In [1]:
!pip install huggingface_hub
!pip install transformers
!pip install tokenizers
!pip install pyctcdecode
import huggingface_hub
import transformers
import tokenizers
print(huggingface_hub.__version__, transformers.__version__, tokenizers.__version__)
!pip install wave

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 5.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.8/416.8 kB 11.4 MB/s eta 0:00:00
0.15.1 4.30.2 0.13.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wave: filename=Wave-0.0.

In [13]:
import torch
import wave
import numpy as np
import scipy.signal as signal
import torchaudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer, AutoModelWithLMHead, AutoTokenizer

# Define the paths for the pre-trained models and tokenizer
model_name = "pongjin/en_with_korean_w2v_model_960h"
tokenizer_name = "pongjin/en_with_korean_w2v_model_960h"
lm_model_name = "gpt2-medium"
lm_tokenizer_name = "gpt2-medium"

# Load the pre-trained models and tokenizer
model = Wav2Vec2ForCTC.from_pretrained(model_name)
tokenizer = Wav2Vec2Tokenizer.from_pretrained(tokenizer_name)
lm_model = AutoModelWithLMHead.from_pretrained(lm_model_name)
lm_tokenizer = AutoTokenizer.from_pretrained(lm_tokenizer_name)

# Set the audio file path
audio_file = "audiofile.wav"

# Open the input WAV file
with wave.open(audio_file, "rb") as wav_in:
    input_sample_rate = wav_in.getframerate()
    audio_data = wav_in.readframes(-1)
    audio_data = np.frombuffer(audio_data, dtype=np.int16)

# Resample the audio data
rate_ratio = 16000 / input_sample_rate
resampled_audio = signal.resample(audio_data, int(len(audio_data) * rate_ratio))

# Create a new WAV file with the resampled audio
with wave.open(audio_file, "wb") as wav_out:
    wav_out.setnchannels(1)
    wav_out.setsampwidth(2)
    wav_out.setframerate(16000)
    wav_out.writeframes(resampled_audio.astype(np.int16).tobytes())

# Load the resampled WAV file
waveform, sample_rate = torchaudio.load(audio_file)

# Set the interval for slicing audio data (in seconds)
interval = 10

# Store sliced audio data
sliced_audio_data = []

# Slice the audio data at the given interval
start_time = 0
end_time = interval * sample_rate
while end_time <= waveform.size(1):
    sliced_audio_data.append(waveform[:, start_time:end_time])
    start_time += interval * sample_rate
    end_time += interval * sample_rate

total_audio = []

#Process the sliced audio data using the Wav2Vec model and LM model
for sliced in sliced_audio_data:
    # Tokenize and encode the input
    input_values = tokenizer(sliced, return_tensors="pt", padding=True, truncation=True).input_values
    input_values = torch.tensor(input_values[0])

    # Pass the input through the Wav2Vec model
    outputs = model(input_values)

    # Perform CTC decoding to obtain predicted text
    predicted_ids = outputs.logits.argmax(dim=-1)
    predicted_text = tokenizer.batch_decode(predicted_ids)[0]

    # Use the LM model to improve accuracy
    lm_input = lm_tokenizer.encode(predicted_text, return_tensors="pt")
    lm_output = lm_model.generate(lm_input)
    completed_text = lm_tokenizer.decode(lm_output[0])

    total_audio.append(completed_text)

print(total_audio)


#     #Use the LM model to improve accuracy
#     lm_input = lm_tokenizer.encode(predicted_text, return_tensors="pt")
#     lm_input = lm_input.to(lm_model.device)  # Ensure LM input is on the same device as the LM model
#     lm_attention_mask = torch.ones_like(lm_input)  # Attention mask for LM model
#     lm_pad_token_id = lm_tokenizer.pad_token_id  # Pad token ID for LM model
#     lm_output = lm_model.generate(
#         input_ids=lm_input,
#         attention_mask=lm_attention_mask,
#         pad_token_id=lm_pad_token_id,
#         max_length=30,
#     )
#     completed_text = lm_tokenizer.decode(lm_output[0])

#     total_audio.append(completed_text)

# print(total_audio)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.


<ipython-input-13-e0d3b9636317>:63: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_values = torch.tensor(input_values[0])
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 29, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 33, but `max_length` is set to 20. This

['sou ofilld garden is the oldest amariden producer of garden pools extrated reputation for quality and relaability has kept it at the', 'top of industry for years of an honrcouncelp for domestic ardening pools vacan rely on southy bardon soufit puch mon mor ou mon', 'pusol mall is efficient with a series of carved blaze w workments braing the blazeclean with a gorden house co after using it to build', 'o your aloan please know that the ma is most effective when i blazer charp bna tossed ablaze and keep children and pass owy from to avoid inlu.', 'ick this mall is intended for usin a lon vegitation such as grass and with using it on hard surfaces such as rock or wood can do wonders', 'band and possibly even brick  blades we understand that from time to time to blade will eventually grow long should the happen.', 'es order another set by following the symple ficue base instruction included with expliet you can easily swich the old ones out for your', 'the new ones well comment bringing 